In [1]:
import re
import requests
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

def crawlingquery(symbol):
    url = f"https://investing.com/search/?q={symbol}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    # 맞는 데이터 찾기
    dom = BeautifulSoup(response.content, "html.parser")
    datas = dom.find_all(class_="js-inner-all-results-quote-item row")
    investing_query = np.nan

    for i in range(len(datas)):
        flag = datas[i].find(class_="flag first")
        try:
            if re.findall('middle (USA)', str(flag))[0] != 'USA':
                continue
        except:
            continue

        investingsymbol = datas[i].find(class_="second").text
        if investingsymbol.upper() != symbol.upper():
            continue

        text = datas[i].find(class_="fourth").text
        if text[:5] != 'Stock':
            continue

        investing_query = re.findall(
            'equities\/([\S]+)', datas[i].get("href"))[0]
        return investing_query

In [2]:
import datetime
def crawling_investing(query):
    try:
        url = f"https://kr.investing.com/equities/{query}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.find_all(
            class_='flex justify-between border-b py-2 desktop:py-0.5')
        # time.sleep(0.5)
        for element in elements:
            try:
                shared = re.findall('발행주식수([0-9\,]+)', element.text)[0]
                shared = int(shared.replace(',', ''))
            except:
                pass

            try:
                sales = re.findall('매출([0-9\.A-Z]+)', element.text)[0]
                sales = float(sales[:-1]) * int(sales[-1].replace('B', str(10000*10000*10)).replace('M', str(10000*100)))
            except:
                pass

            try:
                data = re.findall('다음 수익일자([0-9년월일\s]+)', element.text)[0]
                year = int(re.findall('(2[12])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                date = int(re.findall('([0-9]{1,2})일', data)[0])
                update_date = year*10000 + month*100 + date
                update_date = datetime.datetime.strptime(str(update_date), '%y%m%d') + datetime.timedelta(days=1)
            except:
                continue
            
            
            
        return sales, shared, update_date

    except:
        return np.nan, np.nan, np.nan

In [3]:
def crawling_investing2(query):
    first_date = first_sales = first_profit = first_operating = first_realprofit = second_date = second_sales = second_profit = second_operating = second_realprofit = thirth_date = thirth_sales = thirth_profit = thirth_operating = thirth_realprofit = fourth_date = fourth_sales = fourth_profit = fourth_operating = fourth_realprofit = np.nan
    try:
        url = f"https://kr.investing.com/equities/{query}-financial-summary"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
        response = requests.get(url, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        dates = dom.select("#rsdiv > div > table > thead > tr > th")
        numbers = dom.select("#rsdiv > div > table > tbody > tr > td")
        # time.sleep(0.5)

        for date, i in zip(dates, range(len(dates))):
            if date.text == '기말:':
                data = re.findall('[0-9년월일\s]+', dates[i+1].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                first_date = year*10000 + month*100 + day
                data = re.findall('[0-9년월일\s]+', dates[i+2].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                second_date = year*10000 + month*100 + day
                data = re.findall('[0-9년월일\s]+', dates[i+3].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                thirth_date = year*10000 + month*100 + day
                data = re.findall('[0-9년월일\s]+', dates[i+4].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                fourth_date = year*10000 + month*100 + day
                break

        for number, i in zip(numbers, range(len(numbers))):
            if number.text == '총매출':
                try:
                    first_sales = float(numbers[i+1].text)
                    second_sales = float(numbers[i+2].text)
                    thirth_sales = float(numbers[i+3].text)
                    fourth_sales = float(numbers[i+4].text)
                except:
                    first_sales = second_sales = thirth_sales = fourth_sales = np.nan
            elif number.text == '총 이익':
                try:
                    first_profit = float(numbers[i+1].text)
                    second_profit = float(numbers[i+2].text)
                    thirth_profit = float(numbers[i+3].text)
                    fourth_profit = float(numbers[i+4].text)
                except:
                    first_profit = second_profit = thirth_profit = fourth_profit = np.nan

            elif number.text == '영업 이익':
                try:
                    first_operating = float(numbers[i+1].text)
                    second_operating = float(numbers[i+2].text)
                    thirth_operating = float(numbers[i+3].text)
                    fourth_operating = float(numbers[i+4].text)
                except:
                    first_operating = second_operating = thirth_operating = fourth_operating = np.nan

            elif number.text == '순이익':
                try:
                    first_realprofit = float(numbers[i+1].text)
                    second_realprofit = float(numbers[i+2].text)
                    thirth_realprofit = float(numbers[i+3].text)
                    fourth_realprofit = float(numbers[i+4].text)
                except:
                    first_realprofit = second_realprofit = thirth_realprofit = fourth_realprofit = np.nan
    
        return first_date, first_sales, first_profit, first_operating, first_realprofit
#        dfs1 = [symbol, first_date, first_sales,
#                first_profit, first_operating, first_realprofit]
#        dfs2 = [symbol, second_date, second_sales,
#                second_profit, second_operating, second_realprofit]
#        dfs3 = [symbol, thirth_date, thirth_sales,
#                thirth_profit, thirth_operating, thirth_realprofit]
#        dfs4 = [symbol, fourth_date, fourth_sales,
#                fourth_profit, fourth_operating, fourth_realprofit]

    except:
        return np.nan, np.nan, np.nan, np.nan, np.nan
#        dfs1 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]
#        dfs2 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]
#        dfs3 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]
#        dfs4 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]

#    return dfs1, dfs2, dfs3, dfs4

In [4]:
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests
import time
import re
#from module import crawlingquery, crawling_investing, crawling_investing2

querys = pd.read_csv('../datas/investing_query.csv')

investing = pd.read_csv
today = datetime.now().year % 100 * 10000 + \
    datetime.now().month * 100 + datetime.now().day
df = []
crawling_investings = []
a = 0
for row in querys.values:
    symbol = row[0]
    query = row[1]
    update_date = row[2]
    url = f"https://kr.investing.com/equities/{query}-news"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
    response = requests.get(url, headers=headers)
    if response.status_code >= 300:
        a = 1
        query = crawlingquery(symbol)
        querys['investing_query'][querys['symbol'] == symbol] = query
        url = f"https://kr.investing.com/equities/{query}-news"
        response = requests.get(url, headers=headers)
        if response.status_code >= 300:
            print(f'{symbol}의 서버가 응답하지 않습니다. 에러 코드 {response.status_code}')

    dom = BeautifulSoup(response.content, "html.parser")
    datas = dom.find_all(class_="textDiv")

    for i in range(len(datas)):
        item = datas[i]
        try:
            text = item.select_one('.date').text
            date = re.findall('[0-9]{0,2} 시간 전', text)
        except:
            continue
        title = item.select_one(".title").get("title")
        link = 'https://kr.investing.com' + \
            item.select_one(".title").get("href")
        if date != []:
            df.append({'symbol': symbol, 'title': title,
                       "link": link, 'date': date})
    # time.sleep(0.5)

    if today >= update_date:
        sales, shared, update_date  = crawling_investing(query)
        first_date, first_sales, first_profit, first_operating, first_realprofit = crawling_investing2(query)
        crawling_investings.append([symbol, sales, shared, update_date, first_date, first_sales, first_profit, first_operating, first_realprofit])
        querys['update_date'][querys['symbol'] == symbol] = update_date
        a=1

df = pd.DataFrame(df)
df.to_csv('../datas/investing_news.csv', index=False, header=False, mode='a')

if a == 1:
    querys.to_csv('../datas/investing_query.csv', index=False)

KeyboardInterrupt: 

In [5]:
crawling_investings

[['ACN',
  47950000000.0,
  666947366,
  210922,
  210531,
  13263.8,
  4404.38,
  2118.66,
  1549.43],
 ['KMX',
  23420000000.0,
  162802762,
  210927,
  210531,
  7697.6,
  924.51,
  567.32,
  436.76],
 ['DRI', 7200000000.0, 130842674, 210927, 210530, 2279.2, 539.4, 323.4, 368.5],
 ['FDX',
  78750000000.0,
  265342075,
  210920,
  210228,
  21510.0,
  15094.0,
  1005.0,
  892.0]]

In [ ]:
a

In [ ]:
import pandas as pd
import FinanceDataReader as fdr
import numpy as np
from flask import *
app = Flask(__name__)

def stock_code(val):
    df_sp500 = fdr.StockListing('sp500')
    symbol = df_sp500[df_sp500['Symbol']==val]
    name = symbol.iloc[0,0]
    return name

stock_code('AAPL')
df = fdr.DataReader(stock_code('AAPL'), '2021')

@app.route('/')

@app.route('/home')
def home():
    return "hello worldddddd!"

@app.route('/stock')
def stock():
    return df


if __name__ == '__main__':
    app.run()

In [ ]:
def stock_code(val):
    df_sp500 = fdr.StockListing('sp500')
    symbol = df_sp500[df_sp500['Symbol']==val]
    name = symbol.iloc[0,0]
    return name

stock_code('AAPL')
df = fdr.DataReader(stock_code('AAPL'), '2021')
df


In [ ]:
app = Flask(__name__)

@app.route('/')

@app.route('/home')
def home():
    return "hello worldddddd!"

@app.route('/stock')
def stock_code(val):
    df_sp500 = fdr.StockListing('sp500')
    symbol = df_sp500[df_sp500['Symbol']==val]
    name = symbol.iloc[0,0]
    return name

def stock():
    stock_code('AAPL')
    df = fdr.DataReader(stock_code('AAPL'), '2021')
    df.to_html()


if __name__ == '__main__':
    app.run()